<a href="https://colab.research.google.com/github/idhanth17/SparkCryptoTrend/blob/main/SparkCryptoTrend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install system deps, Spark, Java, and Python packages
# Run these in a Colab code cell (prepend `!`)
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq

# Spark 3.5.0 (matches your earlier code); change version if you prefer
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Python packages
!pip install -q findspark pyspark requests pandas matplotlib seaborn


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,086 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,822 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main

In [ ]:
import os, findspark, time
# Point to Java + Spark (adjust paths if different)
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("CryptoLiveStreaming") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

spark


FileNotFoundError: [Errno 2] No such file or directory: '/content/spark-3.5.0-bin-hadoop3/./bin/spark-submit'

In [ ]:
import requests, pandas as pd, datetime, os

def fetch_binance_tickers(limit_pairs=200):
    urls = [
        "https://api.binance.com/api/v3/ticker/24hr",   # main
        "https://api.binance.us/api/v3/ticker/24hr"     # fallback
    ]

    headers = {"User-Agent": "Mozilla/5.0"}

    data = None
    for url in urls:
        try:
            r = requests.get(url, timeout=15, headers=headers)
            r.raise_for_status()
            data = r.json()
            print(f"✅ Data loaded from: {url}")
            break
        except:
            print(f"⚠️ Failed: {url}, trying next...")

    if data is None:
        raise Exception("❌ Could not fetch data from Binance")

    df = pd.DataFrame(data)
    df = df[df['symbol'].str.endswith("USDT")].head(limit_pairs).copy()
    df = df[["symbol", "lastPrice", "priceChangePercent", "volume", "openTime", "closeTime", "count"]]

    df["lastPrice"] = pd.to_numeric(df["lastPrice"], errors="coerce")
    df["priceChangePercent"] = pd.to_numeric(df["priceChangePercent"], errors="coerce")
    df["volume"] = pd.to_numeric(df["volume"], errors="coerce")

    df["scrape_time"] = datetime.datetime.utcnow().isoformat()

    return df


os.makedirs("/content", exist_ok=True)
df = fetch_binance_tickers(150)
df.to_csv("/content/crypto_live_snapshot.csv", index=False)
print("✅ Saved snapshot: /content/crypto_live_snapshot.csv")

df.head()


⚠️ Failed: https://api.binance.com/api/v3/ticker/24hr, trying next...
✅ Data loaded from: https://api.binance.us/api/v3/ticker/24hr
✅ Saved snapshot: /content/crypto_live_snapshot.csv


/tmp/ipython-input-3047587536.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["scrape_time"] = datetime.datetime.utcnow().isoformat()


,symbol,lastPrice,priceChangePercent,volume,openTime,closeTime,count,scrape_time
6,BTCUSDT,107179.2800,-1.903,2.254079e+01,1762137002383,1762223402383,2146,2025-11-04T02:30:02.628681
7,ETHUSDT,3639.3100,-4.548,2.600272e+02,1762137000237,1762223400237,2407,2025-11-04T02:30:02.628681
8,XRPUSDT,2.3482,-4.715,1.264916e+06,1762137001730,1762223401730,1464,2025-11-04T02:30:02.628681
9,BCHUSDT,511.1000,-3.548,9.218000e+00,1762137002225,1762223402225,116,2025-11-04T02:30:02.628681
10,LTCUSDT,89.8300,-7.487,1.041613e+03,1762137002325,1762223402325,660,2025-11-04T02:30:02.628681


In [ ]:
spark_df = spark.read.csv("/content/crypto_live_snapshot.csv", header=True, inferSchema=True)
spark_df.show(5, truncate=False)
spark_df.printSchema()


+-------+---------+------------------+---------+-------------+-------------+-----+--------------------------+
|symbol |lastPrice|priceChangePercent|volume   |openTime     |closeTime    |count|scrape_time               |
+-------+---------+------------------+---------+-------------+-------------+-----+--------------------------+
|BTCUSDT|107179.28|-1.903            |22.54079 |1762137002383|1762223402383|2146 |2025-11-04 02:30:02.628681|
|ETHUSDT|3639.31  |-4.548            |260.0272 |1762137000237|1762223400237|2407 |2025-11-04 02:30:02.628681|
|XRPUSDT|2.3482   |-4.715            |1264916.2|1762137001730|1762223401730|1464 |2025-11-04 02:30:02.628681|
|BCHUSDT|511.1    |-3.548            |9.218    |1762137002225|1762223402225|116  |2025-11-04 02:30:02.628681|
|LTCUSDT|89.83    |-7.487            |1041.613 |1762137002325|1762223402325|660  |2025-11-04 02:30:02.628681|
+-------+---------+------------------+---------+-------------+-------------+-----+--------------------------+
only showi

In [ ]:
from pyspark.sql.functions import col

# Basic cleaning: remove rows with missing main numeric fields
clean_df = spark_df.filter(
    col("lastPrice").isNotNull() & col("priceChangePercent").isNotNull() & col("volume").isNotNull()
)

# cast types (if needed)
clean_df = clean_df.withColumn("lastPrice", col("lastPrice").cast("double")) \
                   .withColumn("priceChangePercent", col("priceChangePercent").cast("double")) \
                   .withColumn("volume", col("volume").cast("double"))

# Replace any remaining null numeric values with 0
numeric_cols = ["lastPrice", "priceChangePercent", "volume"]
for c in numeric_cols:
    clean_df = clean_df.fillna({c: 0})

print("✅ Cleaned count:", clean_df.count())
clean_df.show(5, truncate=False)


✅ Cleaned count: 150
+-------+---------+------------------+---------+-------------+-------------+-----+--------------------------+
|symbol |lastPrice|priceChangePercent|volume   |openTime     |closeTime    |count|scrape_time               |
+-------+---------+------------------+---------+-------------+-------------+-----+--------------------------+
|BTCUSDT|107179.28|-1.903            |22.54079 |1762137002383|1762223402383|2146 |2025-11-04 02:30:02.628681|
|ETHUSDT|3639.31  |-4.548            |260.0272 |1762137000237|1762223400237|2407 |2025-11-04 02:30:02.628681|
|XRPUSDT|2.3482   |-4.715            |1264916.2|1762137001730|1762223401730|1464 |2025-11-04 02:30:02.628681|
|BCHUSDT|511.1    |-3.548            |9.218    |1762137002225|1762223402225|116  |2025-11-04 02:30:02.628681|
|LTCUSDT|89.83    |-7.487            |1041.613 |1762137002325|1762223402325|660  |2025-11-04 02:30:02.628681|
+-------+---------+------------------+---------+-------------+-------------+-----+-----------------

In [ ]:
# Convert to RDD and run example transforms (like you did)
rdd = clean_df.rdd

# 1) Count number of symbols (per 3-letter base, e.g., BTCUSDT -> BTC)
def base_symbol(row):
    s = row.symbol
    # take everything before 'USDT'
    return (s.replace("USDT",""), 1)

counts_by_base = rdd.map(lambda x: (x.symbol.replace("USDT",""), 1)).reduceByKey(lambda a,b: a+b)
print("Counts by base (take 10):", counts_by_base.take(10))

# 2) Average lastPrice per base
price_avg = (rdd.map(lambda x: (x.symbol.replace("USDT",""), (x.lastPrice, 1)))
                .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
                .mapValues(lambda v: v[0]/v[1]))
print("Avg price per base (take 10):", price_avg.take(10))


Counts by base (take 10): [('BTC', 1), ('ETH', 1), ('XRP', 1), ('BCH', 1), ('LTC', 1), ('BNB', 1), ('ADA', 1), ('BAT', 1), ('ETC', 1), ('XLM', 1)]
Avg price per base (take 10): [('BTC', 107179.28), ('ETH', 3639.31), ('XRP', 2.3482), ('BCH', 511.1), ('LTC', 89.83), ('BNB', 996.32), ('ADA', 0.5601), ('BAT', 0.2061), ('ETC', 15.1), ('XLM', 0.2836)]


In [ ]:
clean_df.createOrReplaceTempView("crypto")

spark.sql("""
SELECT SUBSTRING(symbol, 1, LENGTH(symbol)-4) as base,
       COUNT(*) as pairs,
       AVG(lastPrice) as avg_price,
       AVG(priceChangePercent) as avg_pct_change,
       AVG(volume) as avg_volume
FROM crypto
GROUP BY base
ORDER BY pairs DESC
LIMIT 20
""").show(truncate=False)


+----+-----+---------+--------------+------------+
|base|pairs|avg_price|avg_pct_change|avg_volume  |
+----+-----+---------+--------------+------------+
|BCH |1    |511.1    |-3.548        |9.218       |
|ADA |1    |0.5601   |-5.068        |507925.2    |
|BAT |1    |0.2061   |7.68          |69073.0     |
|DOGE|1    |0.17     |-6.14         |2463456.0   |
|NEO |1    |4.9      |-2.584        |2963.11     |
|VET |1    |0.01481  |-5.428        |5977866.5   |
|QTUM|1    |1.556    |-12.731       |39.0        |
|KNC |1    |0.2733   |-4.072        |924.3       |
|VTHO|1    |0.001168 |10.921        |2.8261444E7 |
|COMP|1    |36.69    |6.968         |30.338      |
|HNT |1    |0.0      |0.0           |0.0         |
|MKR |1    |1750.0   |-4.891        |1.9839      |
|ONE |1    |0.00513  |-9.842        |1.31456297E7|
|BAND|1    |0.448    |-0.885        |839.3       |
|UNI |1    |5.192    |-8.752        |832.57      |
|SOL |1    |167.65   |-9.005        |11322.633   |
|PAXG|1    |3995.65  |0.0      

In [ ]:
from pyspark.sql.functions import when

# Label: trend = 1 if priceChangePercent > 0.5% OR volume > high threshold (tune as needed)
model_df = clean_df.withColumn(
    "trend",
    when((col("priceChangePercent") > 0.5) | (col("volume") > 1e6), 1).otherwise(0)
)

model_df.select("symbol","lastPrice","priceChangePercent","volume","trend").show(10, truncate=False)


+-------+---------+------------------+---------+-----+
|symbol |lastPrice|priceChangePercent|volume   |trend|
+-------+---------+------------------+---------+-----+
|BTCUSDT|107179.28|-1.903            |22.54079 |0    |
|ETHUSDT|3639.31  |-4.548            |260.0272 |0    |
|XRPUSDT|2.3482   |-4.715            |1264916.2|1    |
|BCHUSDT|511.1    |-3.548            |9.218    |0    |
|LTCUSDT|89.83    |-7.487            |1041.613 |0    |
|BNBUSDT|996.32   |-6.268            |423.988  |0    |
|ADAUSDT|0.5601   |-5.068            |507925.2 |0    |
|BATUSDT|0.2061   |7.68              |69073.0  |1    |
|ETCUSDT|15.1     |-4.491            |1130.65  |0    |
|XLMUSDT|0.2836   |-4.222            |308134.0 |0    |
+-------+---------+------------------+---------+-----+
only showing top 10 rows



In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

feature_cols = ["lastPrice","priceChangePercent","volume"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

final_df = assembler.transform(model_df).select("features","trend")

train_df, test_df = final_df.randomSplit([0.7,0.3], seed=42)
rf = RandomForestClassifier(labelCol="trend", featuresCol="features", numTrees=20, maxDepth=5)
rf_model = rf.fit(train_df)

preds = rf_model.transform(test_df)
evaluator = BinaryClassificationEvaluator(labelCol="trend")
auc = evaluator.evaluate(preds)
print(f"✅ Trained RF model — AUC = {auc:.3f}")

# Save model (overwrite if exists)
model_path = "/content/crypto_rf_model"
try:
    rf_model.write().overwrite().save(model_path)
    print("Model saved to:", model_path)
except Exception as e:
    print("Model save error:", e)


✅ Trained RF model — AUC = 1.000
Model saved to: /content/crypto_rf_model


In [ ]:
# We'll write small CSV files into /content/crypto_stream_input periodically and let Spark read them.
import shutil, os
stream_input_dir = "/content/crypto_stream_input"
stream_batch_dir = "/content/crypto_stream_batch"

# Clean and create
shutil.rmtree(stream_input_dir, ignore_errors=True)
shutil.rmtree(stream_batch_dir, ignore_errors=True)
os.makedirs(stream_input_dir, exist_ok=True)
os.makedirs(stream_batch_dir, exist_ok=True)

# Write an initial batch of files (these will be read by Spark streaming)
# We'll create many small files that will be added one-by-one by a producer thread.
sample_pd = model_df.limit(100).toPandas()
# split into many small files for streaming
for i in range(0, 20):
    small = sample_pd.sample(frac=0.2, replace=True)  # shuffle/sample so files look different
    small.to_csv(f"{stream_batch_dir}/batch_{i:03d}.csv", index=False)

print("Prepared", len(os.listdir(stream_batch_dir)), "batch files in", stream_batch_dir)


Prepared 20 batch files in /content/crypto_stream_batch


In [ ]:
# ---------- Cell: Streaming Only (Fixed Version) ----------
import os, time, threading, shutil
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("CryptoStreamOnly").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

batch_dir = "/content/crypto_stream_batch"
stream_dir = "/content/crypto_stream_input"
pred_out = "/content/predictions_only"

# Reset output directory
shutil.rmtree(pred_out, ignore_errors=True)
os.makedirs(pred_out, exist_ok=True)

# Reset streaming input folder
shutil.rmtree(stream_dir, ignore_errors=True)
os.makedirs(stream_dir, exist_ok=True)

# Schema from batch files
schema = spark.read.csv(batch_dir, header=True, inferSchema=True).schema

# Streaming source
stream_df = (
    spark.readStream
    .schema(schema)
    .option("maxFilesPerTrigger", 1)
    .csv(stream_dir)
)

# ✅ Handle NULL values before assembling features
stream_df = stream_df.fillna({"lastPrice":0, "priceChangePercent":0, "volume":0})

# ✅ Assemble features (allow invalid rows)
assembler = VectorAssembler(
    inputCols=["lastPrice", "priceChangePercent", "volume"],
    outputCol="features",
    handleInvalid="keep"
)

stream_features = assembler.transform(stream_df)

# Load Model
model = RandomForestClassificationModel.load("/content/crypto_rf_model")
pred_df = model.transform(stream_features)

# ✅ ONLY select valid CSV-friendly columns
final_preds = pred_df.select(
    "symbol", "lastPrice", "priceChangePercent", "volume", col("prediction").alias("trendPrediction")
)

# Save each batch
# ✅ Save + PRINT each batch
def save_preds(df, epoch):
    # Select only final cols
    pdf = df.select(
        "symbol","lastPrice","priceChangePercent","volume","trendPrediction"
    ).toPandas()

    # Save CSV
    pdf.to_csv(f"{pred_out}/preds_{epoch}.csv", index=False)

    # ✅ PRINT first few rows for visibility
    print(f"\n✅ Saved preds_{epoch}.csv — Showing latest predictions:\n")
    print(pdf.head().to_string(index=False))  # clean print format
    print("-" * 80)

query = (
    final_preds.writeStream
    .foreachBatch(save_preds)
    .outputMode("append")
    .start()
)


# Send rows gradually
def send_rows():
    df = clean_df.limit(200).toPandas()
    for i, row in df.iterrows():
        pd.DataFrame([row]).to_csv(f"{stream_dir}/row_{i}.csv", index=False)
        if i % 10 == 0:
            print(f"📩 Streamed row_{i}.csv")
        time.sleep(2)

threading.Thread(target=send_rows, daemon=True).start()

print("🚀 Streaming Started — No Queries Running")
query.awaitTermination(100)


🚀 Streaming Started — No Queries Running
📩 Streamed row_0.csv

✅ Saved preds_0.csv — Showing latest predictions:

  symbol  lastPrice  priceChangePercent    volume  trendPrediction
  symbol   0.000000               0.000       0.0              0.0
REEFUSDT   0.000213              -1.843 2195905.0              1.0
--------------------------------------------------------------------------------

✅ Saved preds_1.csv — Showing latest predictions:

 symbol  lastPrice  priceChangePercent   volume  trendPrediction
 symbol     0.0000               0.000     0.00              0.0
APEUSDT     0.3737              -6.011 60511.78              0.0
--------------------------------------------------------------------------------

✅ Saved preds_2.csv — Showing latest predictions:

 symbol  lastPrice  priceChangePercent   volume  trendPrediction
 symbol       0.00               0.000  0.00000              0.0
BTCUSDT  107179.28              -1.903 22.54079              0.0
-----------------------------

False

In [ ]:
# ✅ RUN SQL QUERIES SEPARATE BLOCK
from pyspark.sql.functions import *

latest_pred_df = spark.read.csv("/content/predictions_only", header=True, inferSchema=True)

latest_pred_df.createOrReplaceTempView("crypto_live")

print("📌 Running post-stream queries on latest data...\n")


📌 Running post-stream queries on latest data...



In [ ]:
spark.sql("""
SELECT symbol, priceChangePercent, lastPrice, volume
FROM crypto_live
ORDER BY priceChangePercent DESC
LIMIT 10
""").show(truncate=False)


+---------+------------------+---------+-----------+
|symbol   |priceChangePercent|lastPrice|volume     |
+---------+------------------+---------+-----------+
|BTRSTUSDT|49.242            |0.197    |1716.2     |
|STMXUSDT |23.288            |9.0E-5   |1.6775574E7|
|RENUSDT  |17.039            |0.01154  |338990.0   |
|KAVAUSDT |15.669            |0.1469   |79155.1    |
|VTHOUSDT |10.921            |0.001168 |2.8261444E7|
|YFIUSDT  |9.745             |4640.0   |1.10411    |
|ZECUSDT  |8.835             |440.0    |1821.262   |
|BATUSDT  |7.68              |0.2061   |69073.0    |
|COMPUSDT |6.968             |36.69    |30.338     |
|CLVUSDT  |5.772             |0.01869  |46124.6    |
+---------+------------------+---------+-----------+



In [ ]:
spark.sql("""
SELECT symbol, volume, lastPrice
FROM crypto_live
ORDER BY volume DESC
LIMIT 10
""").show(truncate=False)


+---------+-------------+---------+
|symbol   |volume       |lastPrice|
+---------+-------------+---------+
|SHIBUSDT |6.485674115E9|9.24E-6  |
|FLOKIUSDT|2.072518208E9|6.109E-5 |
|XECUSDT  |3.5340996E7  |1.341E-5 |
|JAMUSDT  |3.3475685E7  |3.1E-5   |
|VTHOUSDT |2.8261444E7  |0.001168 |
|STMXUSDT |1.6775574E7  |9.0E-5   |
|ONEUSDT  |1.31456297E7 |0.00513  |
|VETUSDT  |5977866.5    |0.01481  |
|DGBUSDT  |4819509.9    |0.00771  |
|DOGEUSDT |2463456.0    |0.17     |
+---------+-------------+---------+



In [ ]:
spark.sql("""
SELECT trendPrediction AS trend_label, COUNT(*) AS count
FROM crypto_live
GROUP BY trendPrediction
""").show()


+-----------+-----+
|trend_label|count|
+-----------+-----+
|        1.0|   30|
|        0.0|  248|
+-----------+-----+

